# ディープラーニングハンズオン

ディープラーニングの基本的な用語や概念を知らない方は、先に「[これから始める人の為のディープラーニング基礎講座](http://images.nvidia.com/content/APAC/events/deep-learning-institute-jp/2017/pdf/nv-murakami.pdf)」に目を通すことを推奨します。

## 開発環境

このハンズオンでは、GPUの搭載されたDLマシンにアクセスし、DIGITSを用いてモデルの学習を行い、最後には、学習済みモデルを用いて推論を行います。

前半はDLマシンにブラウザからアクセスできれば、演習が行えます。後半は、PCのDocker上にcaffeが動作する環境があることを想定しています。

## 概要
ディープラーニングは、ヒトの視覚に近いレベルの認識をコンピュータで実現する技術です。このハンズオンでは、機械学習のワークフローとともに、現実世界の画像認識問題をディープニューラルネットワークを使って解決することにチャレンジします。私たちは、データの前処理、学習、評価、そして精度向上のためのチューニング作業を含むディープラーニングのすべてのワークフローを体験します。また、モデルを学習する際のGPUの恩恵についても確認することになるでしょう。このハンズオンを完了した際には、自身の画像分類データセットを用いて、ディープニューラルネットワークの学習を行うことのできる知識を身に付けています。

## 1. Hello MNIST
ディープラーニングでは、多くの中間層をもつ人工ニューラルネットワークの学習を行います。近年、ディープラーニングを用いることにより、コンピュータビジョンや自然言語処理をはじめとした様々な分野でめざましい発展を遂げています。

ディープラーニング成功の１つのカギは、畳み込みニューラルネットワーク(Convolutional Neural Network; CNN)です。従来のニューラルネットワークは、前の層と後ろの層のすべてのニューロンが結合した構造(全結合)になっています。CNNでは、局所受容野および重み共有と呼ばれる特別な層間結合を持ちます。生物の脳の視覚野を模倣したCNNは、従来のコンピュータビジョンで行われていたような特徴量の設計を行う必要なく、特徴抽出を可能にしています。

ディープラーニングのワークフローを体験するために、DIGITSを使いましょう。DIGITSはGPUを用いたディープラーニングトレーニングシステムです。CNNの開発やテストを簡単に行うことができます。DIGITSは複数のディープラーニングフレームワーク、データ形式をサポートしており、画像分類や物体検出などの学習に対応しています。

さらに、DIGITSはワークロードマネージャーの機能も含まれています。ワークロードマネージャーはユーザーに異なるジョブを複数走らせることを可能にします。複数のGPUが存在する場合、ジョブを同時に実行することができます。もしジョブ数が使用可能リソースよりも多い場合でも、ジョブはリソースが使用可能になったタイミングまでキューイングされます。DIGITSのダッシュボードではすべてのアクティブなジョブとその状態を確認することができます。

今回のチュートリアルでは、Caffeフレームワークを使用します。CaffeはBerkeley Vision and Learning Center(BVLC)によって作成されたフレームワークで、GPUを用いた高速な処理が可能であり、コードを書くことなくニューラルネットワークの学習が行えるようになっています。モデルの学習はマルチGPUを使って並列化し、高速化することができます。

まずは、手書き文字データベース[MNIST](http://yann.lecun.com/exdb/mnist/)を使って、DIGITSの基本操作を体験しましょう。MNISTには、0から9までの手書きの数字の画像が70,000枚含まれています。MNISTはディープラーニングの入門によく使われています。

### 1-1. DIGITSを始めましょう
DIGITSを開始すると、以下のようなホーム画面を見ることになります。

![](images/digits_home.png)

DIGITSを使い新しいデータセットと新しいモデルを作成することが可能です。このホーム画面には、過去に実行した、または実行中のモデルやデータ作成プロセスがすべて表示されます。デフォルトウィンドウパネルはモデルのビューが表示されるようになっています。もしデータセットを見たい場合は、左にある**Datasets**タブを選択してください。右側には**New Dataset**または**New Model**というタブがあります。このタブを選択することで新しいデータセットや新しいモデルを作成することが可能です。クリックすると選択メニューが表示されます。

![](images/digits_zoom_new_dataset.png)

このチュートリアルでは、**Classification**オプションを使用します。

### 1-2. データセットの作成
まずはじめに、データベースをMNISTデータから作成する必要があります。**New Dataset**メニューから**Classification**を選んでください。このとき、あなたはユーザ名を入力する必要があります。このユーザ名は、あなたを識別できるものであれば、なんでもかまいません。

**New Dataset**ウィンドウでは、以下のフィールドをセットします。
- Image Type : Grayscale
- Image Size : 28 x 28
- Training Images : /usr/local/images/mnist/train
- Select **Separate validation images folder** checkbox
    　　※ **Separate test images folder**は使えません
- Varidation Images : /usr/local/images/mnist/test
- Group Name : [your name]
- Dataset Name : MNIST_[your name]


![](images/digits_new_classification_dataset.png)

フィールドに入力が終わったら**create**ボタンを押してください。

次のウィンドウはジョブの進行状況と完了予想時間を表示します。このジョブは数分かかります。

終わったら、**Create DB(train)**パネルの下の方にある**Explore**ボタンを探してください。

![](images/digits_explore_db.png)

ここで、データベースの中のすべての画像を確認することができます。データベースが作成される際、画像の並びはランダムになります。

これからDIGITSを用いて、このデータを正しく分類したいと思います。

### 1-3. モデルの作成
画像のデータベースができました。それを使ってネットワークの学習をしてみましょう。すべての画面の左上には**DIGITS**というアイコンが見えているはずです。それをクリックすると、ホーム画面に戻ることができます。ここでは、**New Model**メニューの**Classification**を選んでモデルの作成に進みます。

**New Image Classification Model**ページには、学習のための様々な設定オプションがあります。よく使用するものには以下があります。

- **Select Dataset** - データベースから学習に使うものを１つ選択します
- **Training Epochs** - 学習エポック回数を選択する。１エポックとはトレーニングデータに対して1反復することです。エポック回数はデータやモデルに依存して異なってきますが、少なくとも5回、100回以上になることもあります
- **Snapshot Interval** - モデルの状態をファイルに保存する間隔を何エポックか指定します
- **Validation Interval** - バリデーション(評価)データに対してモデルの精度を計算する間隔を何エポックか指定します
- **Random Seed** - 乱数生成器で使うシード値を指定します。
- **Base Learning Rate** - 重みを更新する際の学習率を指定します。モデルの重みは勾配降下法を用いて計算されます。この値は、各イテレーションでのステップサイズを表します。大きい値では、重みは急速に変更されていきますが、モデルが収束しない場合があります。小さい値では、収束に時間がかかります。

DIGITSには、現在３つのネットワークがデフォルトで入っています。LeNetは手書き文字認識のために開発されたCNNです。2012年、ImageNetのコンペティションで優勝したモデルがAlexNetです。AlexNetは従来型の機械学習の代わりにディープラーニングを使用したもので、コンピュータビジョンの世界に革命を起こし、それ以降ImageNetコンペティションのトップはディープニューラルネットワークをベースとしたものになっています。GoogLeNetは、2014年にImageNetコンペティションの画像分類において新たなスタンダードとなったモデルです。

DIGITSは２つのフレームワークをサポートしています。Caffeはそのひとつです。Torchはもうひとつのフレームワークであり、画像分類だけでなく音声認識や自然言語処理にも適したものです。

私たちのモデルを学習するには、以下のオプションをセットする必要があります。
- **MNIST_[your name]**データセットを選択
- **Training Epochs**を10にする
- **Caffe**フレームワークを選択
- **LeNetモ**デルを選択
- **Goup Name**を[your name]にする
- **Model Name**をLeNet\_MNIST\_[your name]する

![](images/digits_create_new_model.png)

これらのオプションをセットしたら、**Create**ボタンを押してください。モデルの学習が開始します。学習中、モデルの統計データは、ウィンドウ内でどんどん更新されていきます。画面中ほどにあるチャートにはあなたの行っている学習の進行状況が表示されています。

![](images/digits_model_loss_accuracy.png)

３つの量は、トレーニングLoss、バリデーションLoss、バリデーションAccuracyです。トレーニングLossとバリデーションLossはエポックが進むごとに減少すべきです。Accuracyとは、バリデーションデータを正しく分類するモデルの性能を示す値です。データポイントの上にマウスオーバーすると、正確な値を確認することができます。このケースだと、最後のエポックでのAccuracyは99%です。ネットワークの初期化はランダム性があるため、あなたの結果は少し異なるかもしれません。

### 1-4. モデルのテスト
それでは、画像を識別する精度をテストしてみましょう。ウィンドウの下の方にスクロールしていくと、1枚の画像または複数の画像をテストすることができます。左側にある**Image Path**に画像パス**/usr/local/images/mnist/test/2/04415.png**というように入力してください。**Show visualizations and statistics**チェックボックスにチェックを入れ、**Classify One**ボタンを押します。数秒後、新しいウィンドウが画像と画像の分類に関する情報とともに表示されます。

![](images/digits_classify_one.png)

このウィンドウ内では、モデルがどれくらい精度良く認識しているかという情報と、各レイヤーに関する情報が含まれています。99.97%の確率で「２」であるという結果を確認することができます。これは正解ですよね。

LeNetネットワークでは、最初の数レイヤーではデータをスケールします。スケールレイヤーとデータレイヤーはオリジナル画像がどのようにスケールされるか、その結果、画像がどのようなものかを示します。conv1レイヤーは5x5のカーネルサイズで20の出力をもっています。conv1のWeightsセクションは5x5のフィルタが20あり、畳み込みレイヤーからの出力画像サイズは24x24です。実際に、conv1 Activationセクションのデータ出力は、**[20 24 24]**とレポートされています。データはCNNを通すと、小さい異なる特徴を獲得していきます。

pool1レイヤーは、カーネルサイズが２で、ストライドは２で、MAX Poolingが設定されています。これは、2x2のパッチサイズで2ピクセルスキップしながら計算していき、その最大値を返すということを意味しています。画像のインプットサイズは24x24で、出力画像サイズは12x12です。ネットワークを通す過程で、データがどのように変換されるのかを確認することができます。

総学習パラメータは431,080個になります。これは大量のパラメータ数だと感じるかもしれませんが、他のネットワークと比較すると、とても小さい値です。AlexNetやGoogLeNetのようなネットワークは、数千万や数億のパラメータをもっています。中には、10億以上のパラメータを持つものもあります。

以上で、DIGITSの基本的な操作を学習しました。次は、より実践的な演習にチャレンジしましょう。

## 2. Chars74K

この章では、手書き文字(英数字)を認識するモデルの作成を通して、データサイエンティストのワークフローを体験します。

今、あなたはタブレット上で書かれた英数字(0-9,A-Z,a-z)の認識精度の検証を依頼されたとしましょう。

### 2-1. データセットの作成

まず、モデルを作成するためのデータセットを探します。すると、[Chars74K](http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/)というデータセットの中に、手書き英数字の画像データセットがありました。EnglishHnd.tgzには、1クラスあたり55枚の画像が用意されています。
展開すると、
```
English/Hnd/Sample001
English/Hnd/Sample002
English/Hnd/Sample003
...
```
以上のようなフォルダに画像ファイルが格納されています。

DIGITSで利用するためには、フォルダをクラス名にする必要があります。これらは**/usr/local/images/Chars74K/Hnd/**に用意してあります。
```
/usr/local/images/Chars74K/Hnd/0
...
/usr/local/images/Chars74K/Hnd/9
/usr/local/images/Chars74K/Hnd/A
...
/usr/local/images/Chars74K/Hnd/Z
/usr/local/images/Chars74K/Hnd/a
...
/usr/local/images/Chars74K/Hnd/z
```
以上のように62クラス(0-9,A-Z,a-z)の分類問題となります。

それでは、先ほどと同じように、データセットを作成しましょう。
今回もLeNetを利用するため、**Image Size**は28x28にしましょう。**Resize Transformation**は文字のアスペクト比を崩さないように**Crop**を選択しましょう。今回は、バリデーションデータが別途用意されていません。データセットのうち19%をバリデーションデータとして利用することにしましょう。(Training:45, Validatoiin:10 /class) 

![](images/chars74k_create_dataset.png)

データセットが作成できたら、中身も確認しておきましょう。

![](images/chars74k_data.png)

#### (注) データセットはテキストファイルを指定することでも作成できます。
**Use Text Files**タブから指定します。

train.txtには、"画像のパス ラベル"の形式で指定します。ただし、ラベルには0から始める整数値を指定します。
```
/path/to/image000.png 0
/path/to/image001.png 1
/path/to/image002.png 2
...
```

labels.txtには、クラス名を順番に記述します。
```
airplane
automobile
...
```

### 2-2. 最初のモデルの作成

まずは、先ほどと同じようにLeNetを試してみましょう。

**Select Dataset**で作成したChars74Kのデータセットを選択しましょう。**Training epochs**はデータ数が少ないので、多めに60に、**Subtract Mean**は文字認識なので、**None**を選択することにしましょう。LeNetでどれほどの精度が出るのか試してみます。

DIGITSでは、最後の全結合層の出力数をがデータセットのクラス数に自動で設定されます。そのため、今回のデータセットに対しても、LeNetの構成を変更することなく、試すことができます。

![](images/chars74k_model.png)

学習終了時点のAccuracyは約65%です。これでは、十分な精度とは言えなさそうです。

### 2-3. モデルのカスタマイズ

LeNetをベースにモデルを以下のようにカスタマイズしてみましょう。
![](images/LeNet_customize.png)

**Clone Job**を押して、モデル作成画面を開きます。そして、**LeNet**の右側の**Customize**をクリックして、ネットワークをカスタマイズします。
![](images/chars74k_clone.png)


まずは、スケールレイヤーを変更しましょう。パラメータ**scale**に0.00390625(=1/256)を指定すると、入力データのピクセル値[0, 255]を[0, 1)にスケーリングします。
```
layer {
  name: "scale"
...
  power_param {
    scale: 0.00390625
  }
}
```

次に、conv1のnum_outputを20から75に、conv2のnum_outputを50から100に変更します。
```
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "scaled"
  top: "conv1"
...
  convolution_param {
    num_output: 75
...
layer {
  name: "conv2"
  type: "Convolution"
  bottom: "pool1"
  top: "conv2"
...
  convolution_param {
    num_output: 100
```

さらに、畳み込み層conv1とconv2の直後にReLU関数を追加しましょう。
```
layer {
  name: "reluC1"
  type: "ReLU"
  bottom: "conv1"
  top: "conv1"
}
```
```
layer {
  name: "reluC2"
  type: "ReLU"
  bottom: "conv2"
  top: "conv2"
}
```

カスタマイズが終わったら、**Visualize**ボタンで可視化して、意図した通りにカスタマイズできたか確認しましょう。
![](images/chars74k_visualize.png)

確認できたら、**Create**ボタンを押してください。
今度はAccuracyが67%になりました。

### 2-4. 層を深くする

次は、モデルをより"ディープ"にして、精度向上を図ってみましょう。

conv1とconv2の直後に畳み込み層を追加してみましょう。ただし、畳み込み層を通した後に画像サイズが変わらないように、カーネルサイズを5、パディングを2に設定します。また、ReLUも追加します。プーリング層のbottomが変わることにも注意してください。
```
layer {
  name: "conv1a"
  type: "Convolution"
  bottom: "conv1"
  top: "conv1a"
  param {
    lr_mult: 1
  }
  param {
    lr_mult: 2
  }
  convolution_param {
    num_output: 75
    kernel_size: 5
    stride: 1
    pad: 2
    weight_filler {
      type: "xavier"
    }
    bias_filler {
      type: "constant"
    }
  }
}
layer {
  name: "reluC1a"
  type: "ReLU"
  bottom: "conv1a"
  top: "conv1a"
}
```
![](images/chars74k_deep.png)

モデルをディープにした分、計算に時間がかかります。

学習が終わると、Accuracyは約70%であり、先ほどの結果とあまり変わりません。モデルを複雑にして、精度向上を図ったはずなのに、期待通りの結果にはなりませんでした。

### 2-5. Data Augmentation

よりディープなモデルの学習には、その分多くのデータを必要とします。ここでは、データを疑似的に増やすことで、精度向上を図ります。

データをよく見ると、文字が左よりに書いてあったり、下の方に書いてあったりします。しかし、文字が書かれている位置は、文字の識別には関係ありません。そこで、32x32の画像から28x28の領域をランダムに切り出すことで、文字が書かれている位置が異なるデータを作り出します。

まず、**Datasets**ウィンドウを開き、Chars74K\_Hnd\_[your name]を選択し、**Clone Job**をクリックします。**Image Size**を32x32に変更します。**Dataset Name**をChars74K\_Hnd32\_[your name]に変更し、**Create**をクリックします。

つぎに、先ほどのモデルから**Clone Job**を押し、データセットを32x32のものに変更し、**Data Tranformations**の**Crop Size**を28に設定します。

![](images/chars74k_crop.png)

今度は、約80%まで精度が向上しました。

#### 問
一般物体認識では、左右反転によるデータ拡張もよく行われている。それには、Dataレイヤーで以下のように設定する。
しかし、今回のタスクでは適していないと考えられる。その理由を考えよ。
```
layer {
  name: "train-data"
  type: "Data"
...
  transform_param {
    mirror: true
  }
...
```

### 2-6. 学習済みモデルの利用

もし、目的のタスクと似たようなタスクで学習したモデル(学習済みモデル)があれば、それを初期値として利用することで、精度を向上させることができます。

Chars74KにはEnglishFntというデータもあり、これはコンピュータのフォントにイタリックやボールドなどを組み合わせた画像(128x128, 1016枚/class)です。今回、われわれの目的は手書き文字を認識させることですが、フォントを認識するタスクはとても良く似たタスクです。
そこで、EnglishFntを認識するモデルを作成し、それを学習済みモデルとして利用しましょう。

まずは、EnglishFntのデータセットを作成します。データは**/usr/local/images/Chars74K/Fnt**に用意してあります。
![](images/chars74k_fnt_dataset.png)

つぎに、EnglishFntのデータセットで2-5と同じ構成のモデルを学習します。学習が終わったら、**Make Pretrained Model**をクリックします。
![](images/digits_make_pretrained_model.png)

今度は、EnglishHndのデータセットで学習します。この際、**Pretrained Networks**から先ほど学習させたPretrained Modelを選びます。
また、学習済みモデルを利用する場合には、すでに良い初期値が得られているため、学習係数を下げて学習を始めることがあります。ハイパーパラメータの異なるいくつかのモデルを試したいときには、DIGITSでは、「,」に続けて値を入力することで、複数のモデルを試すことができます。
![](images/chars74k_pretrained.png)

今回の例では、学習係数が0.01の方が良い結果が得られました。ついに、85%近くまで精度が向上しました。

### 2-7. デプロイ

ここまでモデルの精度を向上させるために、試行錯誤を繰り返してきました。私たちは、はじめは65%程度しかなかった精度を85%近くにまで向上させることに成功しました。

今度は、アプリケーションとして完成させましょう。

最終的なモデルを**Download Model**からダウンロードしてきましょう。
PCのdata/modelというフォルダを作り、その下に展開しましょう。

`$ tar zxvf 20170714-011255-0c1c_epoch_60.0.tar.gz`

Windowsでペイントを起動しましょう。画像サイズを28x28にして、そこに文字を書いて、test.pngという名前でdata/の下に保存します。

![](images/paint.png)

それでは、今書いた文字を先ほど学習させたモデルで推論させてみましょう。

In [4]:
import os
import sys
import caffe
import numpy as np

IMAGE_FILE = 'test.png'
SNAPSHOT = 'model/snapshot_iter_2640.caffemodel'
MODEL_FILE = 'model/deploy.prototxt'
LABELS = []
with open('model/labels.txt') as f:
    for label in f:
        LABELS.append(label)

caffe.set_mode_cpu()
net = caffe.Classifier(MODEL_FILE, SNAPSHOT, channel_swap=[0], image_dims=(28, 28))
input_image = caffe.io.load_image(IMAGE_FILE, color=False)
input_image *= 255
prediction = net.predict([input_image], False)
print LABELS[prediction[0].argmax()]

A



結果はどうでしたか？あなたが書いた通りの結果が出たでしょうか？

どの程度正しい結果が返ってくるでしょうか？また、正解率の低い文字はどんな文字でしょうか？

いろいろ試してみてください。

## まとめ
このチュートリアルでは、ディープラーニング入門を行い、データサイエンティストのワークフローを体験しました。
ディープラーニングの基礎を学び、実際にCaffeやDIGITSを使ってどのように学習ができるのかを学びました。

あなたはすでに、新たなデータセットに対してディープニューラルネットワークの学習を行うことのできる知識を身に付けています。
それを確かめるには、[CIFAR](https://www.cs.toronto.edu/~kriz/cifar.html)データセットの分類にチャレンジしてみるとよいでしょう。CIFARは32x32のカラー画像、6万枚からなるデータセットで、犬、猫、トラックなどのクラスを含んでいます。CIFAR10は10クラスから構成されており、CIFAR100は100クラスから構成されています。